1. give counts etc in dashboard
2. scrapper should print decsreasing counts
3. dashboard summary after every batch
4. after every run summary of got/total.

In [ ]:
import sys
import os

# Add the parent directory of the module to sys.path
sys.path.append(os.path.abspath(r"E:\EmailTool-V2\Barzaan\email_creator_app"))
from database_training.database_process_mutifile import process_files_and_save_output
from database_training.merger_for_pckl import update_pkl_with_csv
import pandas as pd
from database_training.update_db_manually import update_pickle
from email_creation.create_emails import email_creator_app
from database_training.database_process_single_file import process_single_file

import pickle, os
# Get the full path of the script
script_path = os.path.abspath('test-notebook.ipynb')
# Get the directory containing the script
user_path = os.path.dirname(script_path)
print("Base directory:", user_path)
base_path = user_path+"/files/" 
print("Base directory:", base_path)
Source_file_path,Output_files_path = base_path+'database_source_files' ,base_path+'database_output_files'
Output_files_path = base_path+'database_output_files'
new_db_pickle_file_path, old_db_pickle_file_path = base_path+'main_database//'+'new_db.pkl', base_path+'main_database//'+'old_db.pkl'
print("old_db_pickle_file_path directory:", old_db_pickle_file_path)
# Read the pickle file
from pathlib import Path


file_path = Path(new_db_pickle_file_path)
folder_path = file_path.parent
print(folder_path) 


 
with open(new_db_pickle_file_path, "rb") as file:
    new_data_dict = pickle.load(file)  # Load as a dictionary
# Print or use the dictionary
print("New", len(new_data_dict))
# Read the pickle file
with open(old_db_pickle_file_path, "rb") as file:
    data_dict = pickle.load(file)  # Load as a dictionary
# Print or use the dictionary
print("Old", len(data_dict))
''' 
# New 51413
# Old 1048454
New 51889    6 april 9.30pm
Old 1048906

New 51889
Old 1048906

New 52851 8april 1am
Old 1049850

New 53148  8april 8am
Old 1050139
New 53371   12.30pm
Old 1050354

New 53974  3.41 pm
Old 1050945
New 54163
Old 1051130

'''


Base directory: e:\EmailTool-V2\Barzaan\email_creator_app
Base directory: e:\EmailTool-V2\Barzaan\email_creator_app/files/
old_db_pickle_file_path directory: e:\EmailTool-V2\Barzaan\email_creator_app/files/main_database//old_db.pkl
e:\EmailTool-V2\Barzaan\email_creator_app\files\main_database
New 54163
Old 1051130


' \n# New 51413\n# Old 1048454\nNew 51889    6 april 9.30pm\nOld 1048906\n\nNew 51889\nOld 1048906\n\nNew 52851 8april 1am\nOld 1049850\n\nNew 53148  8april 8am\nOld 1050139\nNew 53371   12.30pm\nOld 1050354\n\nNew 53974  3.41 pm\nOld 1050945\n'

In [45]:
import csv
import re

# Input and output file paths
input_file = r'E:\EmailTool-V2\Barzaan\email_creator_app\query_logs.txt'
output_file = 'logs.csv'

# Prepare to store rows
rows = []

# Email regex to extract from RESULTS
email_pattern = re.compile(r"'([^']+@[^']+)'")

with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('|')
        if len(parts) != 3:
            continue  # Skip malformed lines

        date = parts[0].strip()
        query_part = parts[1].strip()
        results_part = parts[2].strip()

        # Extract company name
        company = query_part.replace('QUERY:', '').strip()

        # Extract email if available
        emails = email_pattern.findall(results_part)
        email = emails[0] if emails else ''

        rows.append([date, company, email])

# Write to CSV
with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter='|')
    writer.writerow(['Date', 'company', 'email'])
    writer.writerows(rows)

print(f"Conversion complete. Output saved to {output_file}")

Conversion complete. Output saved to logs.csv


In [82]:
from automatic_email_format.scarpper_manager import extract_email_format, get_email_pattern, extract_emails , select_best_row
from automatic_email_format.get_email_flags import get_flags
data = pd.read_csv(output_file)
print (data.head(3))

   Unnamed: 0                 date  \
0           0  2025-04-04 23:22:13   
1           1  2025-04-04 23:23:14   
2           2  2025-04-04 23:26:43   

                                        company email domain match  
0  massachusetts water resources authority mwra   NaN    NaN   NaN  
1                 massey's diesel and rv repair   NaN    NaN   NaN  
2                                      masshire   NaN    NaN   NaN  


In [83]:
data["email_formats_list"] = data["email"].apply(lambda x: extract_email_format(x) + extract_emails(x))
data["email pattern"] = data["email_formats_list"].apply(lambda email_list: [get_email_pattern(email) for email in email_list])
df =  data[data['match'].isin(['High', 'Medium'])]
df = df.groupby("company", group_keys=False).apply(select_best_row)
df.reset_index(drop=True, inplace=True)
df["company"] = df["company"].str.strip()
df["email pattern"] = df["email pattern"].apply(lambda x: x[0] if x else "NA")
print (df.head(4))


   Unnamed: 0                 date                          company  \
0        3873  2025-04-05 23:43:41                            acgme   
1        3875  2025-04-05 23:43:49  achievement preparatory academy   
2        3876  2025-04-05 23:43:54   achieving reunification center   
3        3879  2025-04-05 23:44:03                     acn millwork   

                      email               domain   match  \
0            jdoe@acgme.org            acgme.org    High   
1  info@achievementprep.org  achievementprep.org  Medium   
2         info@arcenter.org         arcenter.org  Medium   
3       jane.doe@acninc.com           acninc.com  Medium   

           email_formats_list                   email pattern  
0            [jdoe@acgme.org]  FirstInitialLastName@acgme.org  
1  [info@achievementprep.org]                 Unknown Pattern  
2         [info@arcenter.org]                 Unknown Pattern  
3       [jane.doe@acninc.com]   FirstName.LastName@acninc.com  


C:\Users\Admin\AppData\Local\Temp\ipykernel_7784\1046635858.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("company", group_keys=False).apply(select_best_row)


In [85]:
df.to_csv("testlogfile1.csv")
# Apply the function to each company group

In [ ]:

# Split the single column into three
df[['Date', 'company', 'email']] = df[df.columns[0]].str.split('|', expand=True)

# Drop the original combined column (optional)
df.drop(columns=[df.columns[0]], inplace=True)

# Show the result
print(df)
print (df.head(3))
data  = get_flags(df,'email')
data.to_csv(output_file)

                      Date                                            company  \
0      2025-04-04 23:22:13       massachusetts water resources authority mwra   
1      2025-04-04 23:23:14                      massey's diesel and rv repair   
2      2025-04-04 23:26:43                                           masshire   
3      2025-04-04 23:31:12       massachusetts water resources authority mwra   
4      2025-04-04 23:31:42                                            massart   
...                    ...                                                ...   
18043  2025-04-08 15:41:59  university of pennsylvania college of liberal ...   
18044  2025-04-08 15:42:03  university of pennsylvania department of radia...   
18045  2025-04-08 15:42:07    university of pennsylvania health system - gspp   
18046  2025-04-08 15:42:12  university of pennsylvania naval reserve offic...   
18047  2025-04-08 15:42:15  university of pennsylvania naval reserve offic...   

                           

In [50]:
print (data['match'].value_counts())

match
N/A       13190
Low        1939
Medium     1732
High       1187
Name: count, dtype: int64


In [51]:
# Loop through rows where match is high or medium AND email not found in new_data_dict
c = 0
for index, row in data.iterrows():
    if str(row['match']).lower() in ['high', 'medium']:
        email = str(row['email']).lower().strip()
        if email not in new_data_dict:
            print(f"Company: {row['company']}")
            print(f"Email: {row['email']}")
            print(f"Match: {row['match']}")
            print("Status: Email not found in new_data_dict")
            c+=1
            print(c,"-" * 40)

Company: massart
Email:  Massachusetts College of Art and Design uses 7 email formats: 1. first_initial last@massart.edu (64.6%). Enter a name to find & verify an email >>> 
Match: High
Status: Email not found in new_data_dict
1 ----------------------------------------
Company: massena consulting
Email:  Massena Consulting uses 1 email formats: 1. first@massenaconsulting.com (100.0%). Enter a name to find & verify an email >>> 
Match: High
Status: Email not found in new_data_dict
2 ----------------------------------------
Company: massart
Email: last@massart.edu
Match: High
Status: Email not found in new_data_dict
3 ----------------------------------------
Company: massena consulting
Email: first@massenaconsulting.com
Match: High
Status: Email not found in new_data_dict
4 ----------------------------------------
Company: masshire
Email: jane.doe@mass.gov
Match: Medium
Status: Email not found in new_data_dict
5 ----------------------------------------
Company: masshire downtown boston c

In [66]:
comp='electromechanical solution' 
print (new_data_dict[str.lower(comp)])


FirstName.LastName@electro-mechanical.com


In [57]:
from pathlib import Path
import csv
file_paths = Path(new_db_pickle_file_path)


with open(r"E:\EmailTool-V2\Barzaan\email_creator_app\new_database.csv", "w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Key", "Value"])  # Header
    for key, value in new_data_dict.items():
        writer.writerow([key, value])

In [31]:
k ='10th SFG'
k ='1 source medicine'
print (new_data_dict[str.lower(k)], data_dict[str.lower(k)])


FirstInitialLastName@1sourcepartners.com FirstInitialLastName@1sourcepartners.com


In [32]:
# Print the first 5 key-value pairs from the dictionary
for i, (key, value) in enumerate(new_data_dict.items()):
    if i == 5:
        break
    print(key, ":", value)



# Convert list values to normal strings
normalized_dict = {key: value[0] if isinstance(value, list) and len(value) == 1 else value for key, value in new_data_dict.items()}

# Print normalized dictionary
# Print the first 5 key-value pairs from the dictionary
for i, (key, value) in enumerate(normalized_dict.items()):
    if i == 5:
        break
    print(key, ":", value)


print (len(normalized_dict))

# # Save the modified data back to a pickle file (optional)
# with open(new_db_pickle_file_path, "wb") as f:
#     pickle.dump(normalized_dict, f)

10th sfg : LastName@soc.mil
magellan of nevada : FirstInitialLastName@magellanhealth.com
magellan pipe line : FirstName.LastName@7n.com
maggie shops for you : FirstInitialLastName@hu-friedy.com
maggie speaks : FirstName.LastName@7n.com
10th sfg : LastName@soc.mil
magellan of nevada : FirstInitialLastName@magellanhealth.com
magellan pipe line : FirstName.LastName@7n.com
maggie shops for you : FirstInitialLastName@hu-friedy.com
maggie speaks : FirstName.LastName@7n.com
49152


In [10]:
import pickle
import csv

# Load the pickle file
with open(new_db_pickle_file_path, "rb") as pkl_file:
    data = pickle.load(pkl_file)

# Ensure data is a dictionary
if isinstance(data, dict):
    # Write to CSV with UTF-8 encoding
    with open("newdb_output.csv", "w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Key", "Value"])  # Header
        for key, value in data.items():
            writer.writerow([key, value])

    print("CSV file 'output.csv' has been created successfully with UTF-8 encoding.")
else:
    print("The loaded pickle file does not contain a dictionary.")



CSV file 'output.csv' has been created successfully with UTF-8 encoding.


In [ ]:
# import pickle

# # Load the pickle file
# with open(old_db_pickle_file_path, "rb") as pkl_file:
#     data = pickle.load(pkl_file)

# # Ensure data is a dictionary
# if isinstance(data, dict):
#     key_to_delete = 'White Oak Ventures'  # Replace with the actual key you want to remove
    
#     # Remove the key if it exists
#     if key_to_delete in data:
#         del data[key_to_delete]
#         print(f"Deleted key: {key_to_delete}")
#     else:
#         print(f"Key '{key_to_delete}' not found in the dictionary.")

#     # Save the updated dictionary back to the pickle file
#     with open(old_db_pickle_file_path, "wb") as pkl_file:
#         pickle.dump(data, pkl_file)

#     print("Updated pickle file saved.")
# else:
#     print("The pickle file does not contain a dictionary.")


Deleted key: White Oak Ventures
Updated pickle file saved.


In [22]:
# import pickle
# import os

# # Folder containing the pickle files
# folder_path = r"E:\EmailTool-V2\Barzaan\email_creator_app\files"  # Change this to your actual folder path

# # Initialize an empty dictionary to store merged data
# merged_data = {}

# # Loop through all .pkl files in the folder
# for file_name in os.listdir(folder_path):
#     if file_name.endswith(".pkl"):  # Process only .pkl files
#         file_path = os.path.join(folder_path, file_name)
        
#         # Load the pickle file
#         with open(file_path, "rb") as pkl_file:
#             data = pickle.load(pkl_file)
            
#             # Ensure the file contains a dictionary
#             if isinstance(data, dict):
#                 for key, value in data.items():
#                     if key not in merged_data:  # Add only if the key is not already present
#                         merged_data[key] = value
#             else:
#                 print(f"Skipping {file_name}: Not a dictionary.")

# # Save the merged data into a new pickle file
# output_file = os.path.join(folder_path, "merged_data.pkl")
# with open(output_file, "wb") as pkl_file:
#     pickle.dump(merged_data, pkl_file)

# print(f"Merged pickle file saved as: {output_file}")



In [9]:
f = [{'title': 'Contact - MassHire Downtown Boston Career Center', 'link': 'https://masshiredowntownboston.org/contact/', 'snippet': ' Boston Office · 75 Federal Street, 3rd Floor Boston, MA 02110 · 617.399.3100 · RESEA Program: resea@masshiredowntownboston.org · Unemployment Insurance: · 877.626. '}, {'title': 'MassHire Downtown Boston Career Center Information - RocketReach', 'link': 'https://rocketreach.co/masshire-downtown-boston-career-center-profile_b406d202fc226471', 'snippet': ' Email Format · Management · Technology Stack · Competitors\xa0... '}, {'title': 'MassHire Downtown Boston Career Center: Home', 'link': 'https://masshiredowntownboston.org/', 'snippet': ' Empowering Boston Job Seekers & Young Adults 18-24, with Free Cutting-Edge Knowledge and Resources for Your Job Search. '}, {'title': 'American Job Center Finder | CareerOneStop', 'link': 'https://www.careeronestop.org/LocalHelp/AmericanJobCenters/find-american-job-centers-details.aspx?location=Massachusetts&radius=25&centerID=98116&ct=0&y=0&w=0&e=0&locationSelected=0&sortcolumns=Location&sortdirections=ASC&curPage=1&pagesize=10', 'snippet': ' https://masshiredowntownboston.org/. AJC Detail Information. Directions. General Information. Gen Information. E-mail Address, inquiry@masshiredowntownboston. '}, {'title': 'Meet Our Team - MassHire Downtown Boston Career Center', 'link': 'https://masshiredowntownboston.org/aboutus/meet-our-team/', 'snippet': ' 27 Feb 2025  ·  Adalberto Jaimes is a Manager of Economic Opportunity at JVS with over 10 years of financial education and coaching experience. '}]

In [12]:
f[0]['snippet']

' Boston Office · 75 Federal Street, 3rd Floor Boston, MA 02110 · 617.399.3100 · RESEA Program: resea@masshiredowntownboston.org · Unemployment Insurance: · 877.626. '